In [ ]:
import tensorflow as tf
import numpy as np
import os
import sys
sys.path.insert(0, "/Users/xinran.he/GitProjects/mahjong")

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
from log_parser.discard_prediction_parser import parse_discard_prediction
from log_parser.discard_prediction_training_data import generate_tfexample

In [ ]:
input_file = "/Users/xinran.he/GitProjects/mahjong/data/raw/20180101/2018010112gm-00a9-0000-bc388570.txt"
games = parse_discard_prediction(open(input_file, "r").read())

In [ ]:
def print_one_round(one_round):
    print "current hai:",
    for hai in one_round.center_player.hand:
        print hai.char,
    print
    for i, other_player in enumerate(one_round.other_player):
        if len(other_player.discarded_hai) > 0:
            last_hai = other_player.discarded_hai[-1]
            print "player%d claims: %d is_riich: %d discarded:" % (i, last_hai.num_claim, last_hai.is_after_riichi),
        else:
            print "player%d claims: %d is_riich: %d discarded:" % (i, 0, 0), 
        for dh in other_player.discarded_hai:
            print dh.hai.char,
        print
    print "discarded: %s" % (one_round.discarded_hai.char)
    print "================================================"

In [ ]:
with tf.Session() as sess:
    meta_graph_def = tf.saved_model.loader.load(
        sess,
        [tf.saved_model.tag_constants.SERVING],
        '/Users/xinran.he/GitProjects/mahjong/training/v2_1553184126')
    graph = tf.get_default_graph()
    input_tensor = graph.get_tensor_by_name("input:0")
    logits = graph.get_tensor_by_name("model/Transformer/Squeeze:0")
    is_riichi = set()
    for one_round in games[2].one_round:
        center_player = one_round.center_player
        input_tfrecord = generate_tfexample(one_round, len(is_riichi)>0).SerializeToString()
        pred = sess.run(logits, feed_dict={input_tensor: [input_tfrecord]})
        print_one_round(one_round)
        print one_round.center_player.hand[np.argmax(pred)].char
        if center_player.is_riichi:
            is_riichi.add(center_player.context.field)